In [ ]:
#use the feature engineering file and replicate to create a file to run the prediction on
#the models have been built based on October as the target; need November as target and October relabelled as -1

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import os
dateCols = ['date']
train2=pd.read_csv("/home/cdsw/train_plus_not_imputted.csv",parse_dates=dateCols)
train2["TotalRevenue"]=train2.item_cnt_day*train2.item_price
train2["Month"]=train2.date.dt.month
train2["Year"]=train2.date.dt.year
train2['YM'] = train2['Year']*100+train2["Month"]

#create an average per month - see how it changes
sales_per_item_PM= train2.groupby(['item_id', "shop_id",'YM'])['item_cnt_day',"TotalRevenue" ].sum().reset_index()
sales_per_item_PM["AvPrice"]=sales_per_item_PM.TotalRevenue/sales_per_item_PM.item_cnt_day
sales_per_item_PM["id"]=sales_per_item_PM.item_id.astype("str")+"_"+sales_per_item_PM.shop_id.astype("str")

#pivot the data to get to get into a format required for predicting
sales_perM= sales_per_item_PM.pivot_table(index=["id"],columns="YM",values="item_cnt_day", fill_value=0).reset_index() 

In [4]:
#drop  and 201512

sales=sales_perM.drop([sales_perM.columns[36]],axis="columns")
print("was:", sales.columns)
#rename columns so Oct= sales_minus1
df = pd.DataFrame({'M': range(1,35)})
df["Txt"]="SlsMth_min"+df.M.astype("str")
df2=df.sort_values(["M"],ascending=False)
nms2=list(df2.Txt)
nms1=[]
nms1.extend('i')
nms1.extend(nms2)
nms1.extend('T')
sales.columns=nms1
sales.rename(columns={'i': 'ids','T': 'Target'}, inplace=True)
print("is now:", sales.columns)



was: Index([  'id', 201301, 201302, 201303, 201304, 201305, 201306, 201307, 201308,
       201309, 201310, 201311, 201312, 201401, 201402, 201403, 201404, 201405,
       201406, 201407, 201408, 201409, 201410, 201411, 201412, 201501, 201502,
       201503, 201504, 201505, 201506, 201507, 201508, 201509, 201510, 201511],
      dtype='object', name='YM')
is now: Index(['ids', 'SlsMth_min34', 'SlsMth_min33', 'SlsMth_min32', 'SlsMth_min31',
       'SlsMth_min30', 'SlsMth_min29', 'SlsMth_min28', 'SlsMth_min27',
       'SlsMth_min26', 'SlsMth_min25', 'SlsMth_min24', 'SlsMth_min23',
       'SlsMth_min22', 'SlsMth_min21', 'SlsMth_min20', 'SlsMth_min19',
       'SlsMth_min18', 'SlsMth_min17', 'SlsMth_min16', 'SlsMth_min15',
       'SlsMth_min14', 'SlsMth_min13', 'SlsMth_min12', 'SlsMth_min11',
       'SlsMth_min10', 'SlsMth_min9', 'SlsMth_min8', 'SlsMth_min7',
       'SlsMth_min6', 'SlsMth_min5', 'SlsMth_min4', 'SlsMth_min3',
       'SlsMth_min2', 'SlsMth_min1', 'Target'],
      dtype='object')

In [5]:
#average of sales 3,6, 9 months prior
sales["Sls_Av3"]=  sales[["SlsMth_min3","SlsMth_min2","SlsMth_min1"]].mean(axis=1)
sales["Sls_Av6"]=sales.loc[: , "SlsMth_min6":"SlsMth_min1"].mean(axis=1)
sales["Sls_Av9"]=sales.loc[: , "SlsMth_min9":"SlsMth_min1"].mean(axis=1)

#sales trends of last 3,6,9 months
sales['STrend3'] = np.where(sales.SlsMth_min1==0, 0, (sales.SlsMth_min3-sales.SlsMth_min1)/sales.SlsMth_min1)
sales['STrend6'] = np.where(sales.SlsMth_min1==0, 0, (sales.SlsMth_min6-sales.SlsMth_min1)/sales.SlsMth_min1)
sales['STrend9'] = np.where(sales.SlsMth_min1==0, 0, (sales.SlsMth_min9-sales.SlsMth_min1)/sales.SlsMth_min1)

#variance of sales in last 3, 6, 9 months
sales["SVar3"]=sales.loc[:,"SlsMth_min3":"SlsMth_min1"].var(axis=1)
sales["SVar6"]=sales.loc[:,"SlsMth_min6":"SlsMth_min1"].var(axis=1)
sales["SVar9"]=sales.loc[:,"SlsMth_min9":"SlsMth_min1"].var(axis=1)
sales.head()

,ids,SlsMth_min34,SlsMth_min33,SlsMth_min32,SlsMth_min31,SlsMth_min30,SlsMth_min29,SlsMth_min28,SlsMth_min27,SlsMth_min26,...,Target,Sls_Av3,Sls_Av6,Sls_Av9,STrend3,STrend6,STrend9,SVar3,SVar6,SVar9
0,0_54,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10000_25,1,0,0,0,0,0,1,0,1,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10000_26,0,0,0,0,0,0,1,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10000_29,0,0,0,0,0,0,0,0,1,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10000_30,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
#other features require cuts of the original data and then merging

#standard dev of price in the last months
stdprice= train2.groupby(['item_id', "shop_id",'YM'])['item_price'].std().reset_index()
#set value to 0 where NaN
stdprice["item_price"]=stdprice.item_price.fillna(0)
stdprice["ids"]=stdprice.item_id.astype("str")+"_"+stdprice.shop_id.astype("str")
stdprice2=stdprice.loc[stdprice.YM.isin([201509,201510,201511])]
stdprice3=stdprice2.pivot_table(index=["ids"],columns="YM", values="item_price", fill_value=0).reset_index()
stdprice3.rename(columns={201509:"PStdMinus2",201510:"PStdMinus1",201511:"PStdInMonth"},inplace=True)

#merge to dataset
s1=pd.merge(sales, stdprice3, how="left", on="ids")

In [8]:
#total sales for this shop over last couple of months
#groupby YM, shop,sales
shops= train2.groupby(["YM","shop_id"])["item_cnt_day"].sum().reset_index()
#filter
shops2=shops.loc[shops.YM.isin([201509,201510])]
#pivot
shops3=shops2.pivot_table(index=["shop_id"],columns="YM",values="item_cnt_day",fill_value=0).reset_index()
shops3.columns=["shop_id","ShopSales_Minus2","ShopSales_Minus1"]

In [9]:
#merge this to the dataset
#split the tuple to get the lookup value
s1['shop_id'] = s1['ids'].str.split('_').str[1].astype("int64")
s1["item_id"]=s1["ids"].str.split("_").str[0].astype("int64")
#merge to the shop id
s2=pd.merge(s1, shops3,how="left", on="shop_id")

In [10]:
#total sales for this product over last couple of months
#groupby YM and prod
prods=train2.groupby(["YM","item_id"])["item_cnt_day"].sum().reset_index()
#filter
prods2=prods.loc[prods.YM.isin([201509,201510])]
#pivot
prods3=prods2.pivot_table(index=["item_id"],columns="YM",values="item_cnt_day", fill_value=0).reset_index()
#rename
prods3.columns=["item_id","ProdSalesMinus2","ProdSalesMinus1"]

In [11]:
#merge to the data set
s3=pd.merge(s2,prods3, how="left",on="item_id")

In [12]:
#consistency of sales - continuous or stacatto- is there something every day or everynow and then?
#do a count of number of days where occurs
#group by YM item id and date
consistency=train2.groupby(["YM","date", "item_id"])["item_cnt_day"].sum().reset_index()
#filter to last 3 months
consistency2=consistency.loc[consistency.YM.isin([201508,201509,201510])]
#group by date and item id - count!
consistency3=consistency2.groupby(["YM","item_id"])["date"].count().reset_index()
#pivot
consistency4=consistency3.pivot_table(index=["item_id"],columns="YM",values="date",fill_value=0).reset_index()
#rename
consistency4.columns=["item_id","DaysSoldMinus3","DaysSoldMinus2","DaysSoldMinus1"]

In [13]:
#merge
s4=pd.merge(s3,consistency4,how="left",on="item_id")

In [14]:
#days since first sale
import datetime as dt
#sort consistency by date
firstsale=consistency.sort_values("date")
#drop_duplicates
firstsale2=firstsale.drop_duplicates(subset=["item_id"],keep="first")
firstsale3=firstsale2.drop(["YM","item_cnt_day"],axis=1)
#calculate the number days since sales began

firstsale3["D_Since_FirstSale"]= (dt.datetime.strptime("2015-11-01", "%Y-%m-%d")-firstsale3.date).dt.days

s5=pd.merge(s4,firstsale3[["item_id","D_Since_FirstSale"]],how="left",on="item_id")

In [15]:
#Na's introduced - go through and understand why
#in shop sales some shops didn't sell any - set to zero
s5["ShopSales_Minus2"]=s5["ShopSales_Minus2"].fillna(0)
s5["ShopSales_Minus1"]=s5["ShopSales_Minus1"].fillna(0)
s5["ProdSalesMinus2"]=s5["ProdSalesMinus2"].fillna(0)
s5["ProdSalesMinus1"]=s5["ProdSalesMinus1"].fillna(0)
s5["DaysSoldMinus3"]=s5["DaysSoldMinus3"].fillna(0)
s5["DaysSoldMinus2"]=s5["DaysSoldMinus2"].fillna(0)
s5["DaysSoldMinus1"]=s5["DaysSoldMinus1"].fillna(0)
s5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 424124 entries, 0 to 424123
Data columns (total 58 columns):
ids                  424124 non-null object
SlsMth_min34         424124 non-null int64
SlsMth_min33         424124 non-null int64
SlsMth_min32         424124 non-null int64
SlsMth_min31         424124 non-null int64
SlsMth_min30         424124 non-null int64
SlsMth_min29         424124 non-null int64
SlsMth_min28         424124 non-null int64
SlsMth_min27         424124 non-null int64
SlsMth_min26         424124 non-null int64
SlsMth_min25         424124 non-null int64
SlsMth_min24         424124 non-null int64
SlsMth_min23         424124 non-null int64
SlsMth_min22         424124 non-null int64
SlsMth_min21         424124 non-null int64
SlsMth_min20         424124 non-null int64
SlsMth_min19         424124 non-null int64
SlsMth_min18         424124 non-null int64
SlsMth_min17         424124 non-null int64
SlsMth_min16         424124 non-null int64
SlsMth_min15         424124 

In [17]:
s5.to_csv("/home/cdsw/Novdata_for_preds.csv")